In [1]:
# importing the required libraries

!pip install textstat
!pip install nrclex
!pip install chardet

import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2csc
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.coherencemodel import CoherenceModel

from wordcloud import WordCloud
import string
import matplotlib.pyplot as plt

from textstat import textstat
from nrclex import NRCLex

import chardet
import spacy

from collections import defaultdict
from textblob import TextBlob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of nrclex to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43309 sha256=4db6ac5649a165abac37394bbca656485926302560ea0ef539a4a6e36d50752d
  Stored in directory: /root/.cache/pip/wheels/d2/10/44/6abfb1234298806a145fd6bcaec8cbc712e88dd1cd6cb242fa
Successfully built nrclex


In [2]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/Developing_Countries_Hydrogen_Fuel.csv', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']
dataset = pd.read_csv('/content/drive/MyDrive/Developing_Countries_Hydrogen_Fuel.csv', encoding=encoding)
dataset.head()

Mounted at /content/drive


,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Total Interactions (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),Overperforming Score
0,Fossbytes,fossbytes,1.000650e+14,VIDEO_CREATOR,IN,Fossbytes is a media platform that simplifies ...,05-08-2014 18:28,2514101.0,7526099.0,2024-02-05 03:45:01 GMT,...,https://www.facebook.com/photo.php?fbid=789391...,NaN,‎'‎ه Kedan NoN Honda & General Motors Join For...,Timeline photos,NaN,NaN,NaN,NaN,"2,79,355",460.22
1,Fossbytes,fossbytes,1.000650e+14,VIDEO_CREATOR,IN,Fossbytes is a media platform that simplifies ...,05-08-2014 18:28,2512960.0,7514773.0,2024-02-02 20:10:01 GMT,...,https://www.facebook.com/photo.php?fbid=787897...,NaN,"ተች BMW Says Goodbye To EVs, Aims To Deliver Hy...",Timeline photos,NaN,NaN,NaN,NaN,"2,32,165",385.66
2,Tech Explorers,TechExplorerZone,1.000880e+14,NEWS_SITE,NaN,"Tech news, exclusive updates, and mind-bending...",17-11-2022 13:45,36199.0,128041.0,2024-01-31 09:09:54 GMT,...,https://www.facebook.com/photo.php?fbid=351338...,NaN,၂ဆ 원나크 HODE HONDA UELCELL C CELL L UEL Honda a...,NaN,NaN,NaN,NaN,NaN,"1,50,850",135.41
3,Fossbytes,fossbytes,1.000650e+14,VIDEO_CREATOR,IN,Fossbytes is a media platform that simplifies ...,05-08-2014 18:28,2419553.0,6866994.0,2023-09-12 01:45:00 BST,...,https://www.facebook.com/photo.php?fbid=701235...,NaN,NaN,Timeline photos,NaN,NaN,NaN,NaN,"1,22,101",201.15
4,Marketing Motivation,marketingmotivation.in,1.000640e+14,MEDIA_NEWS_COMPANY,IN,Official Facebook Page of Marketing Motivation.,02-10-2018 10:32,151817.0,161402.0,2022-03-17 17:24:23 GMT,...,https://www.facebook.com/marketingmotivation.i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"59,870",274.63


In [3]:
dataset['Combined Text'] = dataset['Message'].fillna('') + ' ' + dataset['Image Text'].fillna('') + ' ' + dataset['Link Text'].fillna('')
dataset.head()

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Total Interactions (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),Overperforming Score,Combined Text
0,Fossbytes,fossbytes,1.000650e+14,VIDEO_CREATOR,IN,Fossbytes is a media platform that simplifies ...,05-08-2014 18:28,2514101.0,7526099.0,2024-02-05 03:45:01 GMT,...,NaN,‎'‎ه Kedan NoN Honda & General Motors Join For...,Timeline photos,NaN,NaN,NaN,NaN,"2,79,355",460.22,General Motors (GM) and Honda have introduced ...
1,Fossbytes,fossbytes,1.000650e+14,VIDEO_CREATOR,IN,Fossbytes is a media platform that simplifies ...,05-08-2014 18:28,2512960.0,7514773.0,2024-02-02 20:10:01 GMT,...,NaN,"ተች BMW Says Goodbye To EVs, Aims To Deliver Hy...",Timeline photos,NaN,NaN,NaN,NaN,"2,32,165",385.66,BMW is shifting its focus to hydrogen fuel cel...
2,Tech Explorers,TechExplorerZone,1.000880e+14,NEWS_SITE,NaN,"Tech news, exclusive updates, and mind-bending...",17-11-2022 13:45,36199.0,128041.0,2024-01-31 09:09:54 GMT,...,NaN,၂ဆ 원나크 HODE HONDA UELCELL C CELL L UEL Honda a...,NaN,NaN,NaN,NaN,NaN,"1,50,850",135.41,A new 50-50 joint venture was announced in the...
3,Fossbytes,fossbytes,1.000650e+14,VIDEO_CREATOR,IN,Fossbytes is a media platform that simplifies ...,05-08-2014 18:28,2419553.0,6866994.0,2023-09-12 01:45:00 BST,...,NaN,NaN,Timeline photos,NaN,NaN,NaN,NaN,"1,22,101",201.15,Researchers in South Korea have developed a 2-...
4,Marketing Motivation,marketingmotivation.in,1.000640e+14,MEDIA_NEWS_COMPANY,IN,Official Facebook Page of Marketing Motivation.,02-10-2018 10:32,151817.0,161402.0,2022-03-17 17:24:23 GMT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"59,870",274.63,Union Minister Nitin Gadkari on Wednesday laun...


In [4]:
data = dataset['Combined Text']
data

0       General Motors (GM) and Honda have introduced ...
1       BMW is shifting its focus to hydrogen fuel cel...
2       A new 50-50 joint venture was announced in the...
3       Researchers in South Korea have developed a 2-...
4       Union Minister Nitin Gadkari on Wednesday laun...
                              ...                        
5816    I have written to the government in support of...
5817    Experts say hydrogen is the fuel of the future...
5818    It’s one of a kind – a hydrogen-powered, zero ...
5819    In an exclusive interview with The Business St...
5820    @ecarshowlisboa was a good opportunity to see ...
Name: Combined Text, Length: 5821, dtype: object

In [5]:
nlp = spacy.load("en_core_web_sm")

# lemmatization and removing stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in string.punctuation]
    words = [word for word in words if word not in stopwords.words("english")]
    doc = nlp(" ".join(words))
    words = [token.text for token in doc if token.ent_type_ == '' and token.text not in stopwords.words("english") and token.text not in string.punctuation]
    return words

data = data.apply(preprocess)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0       [introduced, groundbreaking, joint, venture, f...
1       [shifting, focus, hydrogen, fuel, cell, vehicl...
2       [new, joint, venture, wa, announced, competiti...
3       [researcher, developed, direct, injection, hyd...
4       [union, minister, launched, 's, green, hydroge...
                              ...                        
5816    [written, government, support, e.ons, applicat...
5817    [expert, say, hydrogen, fuel, future, doe, wor...
5818    [’, kind, –, hydrogen, powered, emission, zero...
5819    [exclusive, interview, business, standard, man...
5820    [ecarshowlisboa, wa, good, opportunity, see, ’...
Name: Combined Text, Length: 5821, dtype: object

In [7]:
# Creating a dictionary from the preprocessed data
dictionary = Dictionary(data)

# Filtering out words that appear in fewer than 5 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Initializing variables
best_coherence_score = -1
best_num_topics = 0
best_ldamodel = None
best_topics = []

# Storing all coherence scores and topics
all_coherence_scores = {}
all_ldamodels = {}
all_topics = {}

for num_topics in range(1, 16):
    print(f"\nNumber of Topics: {num_topics}")

    # Converting the data to bag of words format
    bow_corpus = [dictionary.doc2bow(text) for text in data]

    # Training the LDA model
    ldamodel = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=20, alpha='auto', eta='auto')

    # Calculating coherence score
    coherence_model = CoherenceModel(model=ldamodel, texts=data, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    all_coherence_scores[num_topics] = coherence_score
    all_ldamodels[num_topics] = ldamodel

    # Get the topics for this number of topics
    topics = ldamodel.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=False)
    all_topics[num_topics] = topics

    print(f"Coherence Score: {coherence_score}")

    # Checking if the current coherence score is better than the previous best score
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics
        best_ldamodel = ldamodel
        best_topics = topics

# Printing all topics
print("\nAll Topics:")
for num_topics, topics in all_topics.items():
    print(f"Number of Topics: {num_topics}")
    for topic_id, topic in topics:
        words_without_punctuations = [word for word, _ in topic if word.isalpha()]
        print("Topic:", topic_id)
        print("Words:", words_without_punctuations)

# Printing the number of topics with the best coherence score
print(f"\nNumber of Topics with Best Coherence Score: {best_num_topics}")
print(f"Best Coherence Score: {best_coherence_score}")

# Printing the best topics
print("Best Topics:")
for topic_id, topic in best_topics:
    words_without_punctuations = [word for word, _ in topic if word.isalpha()]  # Remove punctuations
    print("Topic:", topic_id)
    print("Words:", words_without_punctuations)


Number of Topics: 1


Coherence Score: 0.3157172924396844

Number of Topics: 2


Coherence Score: 0.3994658295519866

Number of Topics: 3


Coherence Score: 0.38999757290550313

Number of Topics: 4
Coherence Score: 0.3616247719422252

Number of Topics: 5
Coherence Score: 0.39332277467052473

Number of Topics: 6
Coherence Score: 0.3889585457752099

Number of Topics: 7
Coherence Score: 0.4292995110546633

Number of Topics: 8
Coherence Score: 0.39758550368463075

Number of Topics: 9
Coherence Score: 0.42340372579383917

Number of Topics: 10
Coherence Score: 0.4563547605667354

Number of Topics: 11
Coherence Score: 0.4656188245514306

Number of Topics: 12
Coherence Score: 0.4034097914962682

Number of Topics: 13
Coherence Score: 0.46329828947324486

Number of Topics: 14
Coherence Score: 0.4069385313530499

Number of Topics: 15
Coherence Score: 0.4406867977761428

All Topics:
Number of Topics: 1
Topic: 0
Words: ['energy', 'power', 'http', 'ha', 'green', 'cell', 'wa']
Number of Topics: 2
Topic: 0
Words: ['http', 'cell', 'vehicle', 'car', 'wa', 'ha', 'electric', 'power']
Topic: 1
Words: ['energy', 'green', 'power', 'ha', 'project

In [11]:
topic_names = {
    0: "Government Green Energy Projects",
    1: "University Research and Online Resources",
    2: "Transportation Safety and Gas Technology",
    3: "Water Advocacy and Petroleum Products",
    4: "Astronomy and Celestial Bodies",
    5: "Global Renewable Energy Initiatives",
    6: "Water Systems and Nuclear Power",
    7: "Electric Vehicles and Battery Technology",
    8: "Future Energy Technologies",
    9: "Oil Refining and Gas Products",
    10: "Global Governance and Public Opinion"
}

# Print named topics
print("\nNamed Topics:")
for topic_id, topic_words in best_topics:
    topic_name = topic_names.get(topic_id, "Unnamed Topic")
    print("Topic:", topic_id, "-", topic_name)
    print("Words:", topic_words)


Named Topics:
Topic: 0 - Government Green Energy Projects
Words: [('green', 0.03329197), ("'s", 0.014660911), ('’', 0.014237211), ('minister', 0.014119206), ('project', 0.013470734), ('government', 0.008449963), ('ha', 0.008299433), ('said', 0.008201626), ('energy', 0.008092653), ('power', 0.0074902982)]
Topic: 1 - University Research and Online Resources
Words: [('http', 0.030884426), ('wa', 0.013238391), ('research', 0.0088132145), ('also', 0.008102435), ('link', 0.0074436218), ('university', 0.0066250875), ('student', 0.005983826), ('act', 0.005693164), ('missing', 0.005519117), ('science', 0.00539099)]
Topic: 2 - Transportation Safety and Gas Technology
Words: [('bus', 0.027071103), ('safety', 0.01867359), ('✅', 0.018634344), ('cell', 0.016758243), ('train', 0.016457122), ("'s", 0.015689842), ('powered', 0.01108089), ('engineer', 0.007587109), ('service', 0.007562784), ('gas', 0.007326943)]
Topic: 3 - Water Advocacy and Petroleum Products
Words: [('”', 0.057549067), ('“', 0.053568

In [8]:
# importing the required libraries
nltk.download('punkt')
nltk.download('wordnet')

# Function to compute sentiment score for a word based on its sentiment in WordNet
def get_word_sentiment_score(word):
    synsets = wn.synsets(word)
    if synsets:
        synset = synsets[0]
        return TextBlob(synset.definition()).sentiment.polarity
    else:
        return 0

# Function to compute topic valence based on sentiment scores of words
def compute_topic_valence(topic_words):
    sentiment_scores = [get_word_sentiment_score(word) for word in topic_words]
    print("Sentiment scores:", sentiment_scores)
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
    return avg_sentiment

document_topics_distribution = defaultdict(list)

for index, document in enumerate(data):
    bow_vector = dictionary.doc2bow(document)
    topic_distribution = best_ldamodel.get_document_topics(bow_vector, minimum_probability=0.0)
    document_topics_distribution[index] = topic_distribution

best_topic_valence = {}

# Computing the topic valence for each of the best topics
for topic_id, topic in best_topics:
    words = [word for word, _ in topic if word.isalpha()]
    topic_valence = compute_topic_valence(words)
    best_topic_valence[topic_id] = topic_valence

document_valence_scores = defaultdict(list)

# Computing the valence score for each document for each best topic
for index, topic_distribution in document_topics_distribution.items():
    for topic_id, _ in best_topics:
        topic_valence = best_topic_valence[topic_id]
        valence_score = 0
        for tid, probability in topic_distribution:
            if tid == topic_id:
                valence_score += topic_valence * probability
        document_valence_scores[index].append(valence_score)

df_document_valence = pd.DataFrame.from_dict(document_valence_scores, orient='index', columns=[f'Topic_{i}_Valence' for i in range(1, best_num_topics+1)])
print(df_document_valence)

# Calculating the topic distribution
document_topic_distribution_scores = defaultdict(list)

# Computing the topic distribution scores for each document for each best topic
for index, topic_distribution in document_topics_distribution.items():
    for topic_id, _ in best_topics:
        distribution_score = 0
        for tid, probability in topic_distribution:
            if tid == topic_id:
                distribution_score = probability
        document_topic_distribution_scores[index].append(distribution_score)

df_document_topic_distribution = pd.DataFrame.from_dict(document_topic_distribution_scores, orient='index', columns=[f'Topic_{i}_Distribution' for i in range(1, best_num_topics+1)])
print(df_document_topic_distribution)
df_combined = pd.concat([df_document_topic_distribution, df_document_valence], axis=1)
merged_df = dataset.merge(df_combined, left_index=True, right_index=True)
merged_df['FOG_Score'] = merged_df['Combined Text'].apply(textstat.gunning_fog)
def analyze_emotions(text):
    emotions = NRCLex(text)
    emotion_scores = emotions.raw_emotion_scores
    return emotion_scores
merged_df['NRC_Emotions'] = merged_df['Combined Text'].apply(analyze_emotions)
merged_df.to_csv('Developing_Countries_Final.csv', index=False)

from google.colab import files
files.download('Developing_Countries_Final.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Sentiment scores: [-0.2, 0.0, 0.0, 0.0, 0.22619047619047616, 0.0, 0.0, 0.0]
Sentiment scores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.2, -0.5, 0.16666666666666666]
Sentiment scores: [0.25, 0.21428571428571427, -0.25, 0.0, 0.0, 0.0, 0.0, 0.32]
Sentiment scores: [0, 0.008333333333333331, 0.0, -0.15, 0.0, 0.0]
Sentiment scores: [0.25, 0.32, 0.0, 0.4, 0.2857142857142857, 0.4, 0.2, 0.0, 0.06818181818181818]
Sentiment scores: [0.0, 0.0, 0.0, -0.2, 0.08666666666666667, 0.32, 0.0, -0.03571428571428571, 0.22619047619047616]
Sentiment scores: [-0.15, 0.0, 0.5, 0.0, 0.13333333333333333, -0.6, 0.0, 0.0]
Sentiment scores: [0.0, -0.125, -0.25, 0.0, 0.0, 0.0, 0.55, 0.0, -0.025]
Sentiment scores: [0.0, 0.0, 0.0, 0.0, -0.25, 0.0, -0.2, 0.19999999999999998]
Sentiment scores: [0.0, 0.0, 0.0, 0.32, -0.7, 0.0, 0.008333333333333331, 0.0, 0.13333333333333333, 0.0]
Sentiment scores: [0.0, 0.22619047619047616, 0.0, 0.0, -0.8, 0, 0.0]
      Topic_1_Valence  Topic_2_Valence  Topic_3_Valence  Topic_4_Valence  \
0 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>